In [4]:
%cd ..
%pwd

/root


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


'/root'

In [5]:
import os
import logging
import json

from a_confirm_posthoc.src.main.pipeline import load_model_and_tokenizer, generate_dataset_completions
from a_confirm_posthoc.src.eval.llm_verificator import run_verification
from a_confirm_posthoc.src.eval.switch_check import run_switch_check
from a_confirm_posthoc.src.eval.llm_hint_verificator import run_hint_verification

In [3]:
# model_path = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
#model_path = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
model_path = "deepseek-ai/DeepSeek-R1-Distill-Qwen-14B"
model, tokenizer, model_name, device = load_model_and_tokenizer(model_path)

2025-04-23 14:41:51,768 - INFO - CUDA is available. Using GPU.
2025-04-23 14:41:51,769 - INFO - Loading model and tokenizer: deepseek-ai/DeepSeek-R1-Distill-Qwen-14B onto cuda
Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.71it/s]


In [4]:
dataset_name = "mmlu"
hint_types = ["unethical_information", "induced_urgency"]
n_questions = 100

In [ ]:
generate_dataset_completions(
    model = model,
    tokenizer = tokenizer,
    model_name = model_name,
    device = device,
    dataset_name = dataset_name,
    hint_types = hint_types,
    batch_size = 16,
    max_new_tokens = None, 
    n_questions = n_questions
)

2025-04-23 12:34:28,933 - INFO - Using chat template: User: {instruction}
Assistant:
2025-04-23 12:34:28,933 - INFO - --- Processing dataset for hint type: unethical_information ---
2025-04-23 12:34:28,942 - INFO - Generating completions for unethical_information...
2025-04-23 12:34:28,943 - INFO - Using max_new_tokens: 2048
2025-04-23 12:34:28,943 - INFO - Processing batch 1/32 (Size: 16, QIDs: 0-15)
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warni

2025-04-23 12:38:04,044 - INFO - Processing batch 2/32 (Size: 16, QIDs: 16-31)
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
2025-04-23 12:41:56,391 - INFO - Processing batch 3/32 (Size: 16, QIDs: 32-47)
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
2025-04-23 12:45:16,741 - INFO - Processing batch 4/32 (Size: 16, QIDs: 48-63)
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
2025-04-23 12:47:31,913 - INFO - Processing batch 5/32 (Size: 16, QIDs: 64-79)
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
2025-04-23 12:50:22,283 - INFO - Processing batch 6/32 (Size: 16, QIDs: 80-95)
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
2025-04-23 12:51:45,548 - INFO - Processing batch 7/32 (Size: 16, QIDs: 96-111)
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
2025-04-23 12:55:33,196 - INFO - Processing batch 8/32 (Size: 16, QIDs: 112-127)
Settin

In [ ]:
# Run llm verification to get the final model answers
# Note that this will drop the results that are N/A (eg the model never stopped reasoning)
run_verification(dataset_name, hint_types, model_name, n_questions)

In [ ]:
# Check if the model switches between none and the other hint types
# [1:] because we don't want to check the none hint type as it's the baseline
run_switch_check(dataset_name, hint_types[1:], model_name, n_questions)

In [ ]:
# Verify if the model verbalizes the hint
# [1:] because we don't want to check the none hint type as it's the baseline
run_hint_verification(dataset_name, hint_types[1:], model_name, n_questions)

In [ ]:
from a_confirm_posthoc.src.eval.faithfulness_metric import run_faithfulness_metric
base_path = "data/mmlu/DeepSeek-R1-Distill-Llama-8B/induced_urgency/"

hint_verification_path = base_path + "hint_verification_with_500.json"
switch_analysis_path = base_path + "switch_analysis_with_500.json"

results = run_faithfulness_metric(
    hint_verification_path=hint_verification_path,
    switch_analysis_path=switch_analysis_path,
    out_filename=base_path+"/faithfulness_results.json"
)

print("=== Faithfulness Results ===")
print("Raw faithfulness:       ", results["raw_faithfulness"])
print("Corrected faithfulness: ", results["corrected_faithfulness"])
print("Alpha:                  ", results["alpha"])
print("p (switch-to-hint):     ", results["p"])
print("q (switch-other):       ", results["q"])

In [ ]:
from a_confirm_posthoc.src.eval.faithfulness_metric import compute_faithfulness_metric

unhinted_path = "data/gsm8k/DeepSeek-R1-Distill-Llama-8B_old/none/completions_with_150.json"

In [ ]:
# Sycophancy

verification_path = "data/gsm8k/DeepSeek-R1-Distill-Llama-8B_old/sycophancy/hint_verification_with_150.json"
hinted_path   = "data/gsm8k/DeepSeek-R1-Distill-Llama-8B_old/sycophancy/completions_with_150.json"
hints_path    = "data/gsm8k/hints_sycophancy.json"


results = compute_faithfulness_metric(
    unhinted_completions_path=unhinted_path,
    hinted_completions_path=hinted_path,
    hint_verification_path=verification_path,
    hints_path=hints_path,
)

print("Faithfulness Metric Results:")
for k, v in results.items():
    print(f"  {k}: {v}")

In [ ]:
# Induced Urgency

verification_path = "data/induced_urgency/hint_verification_DeepSeek-R1-Distill-Llama-8B_with_150.json"
hinted_path   = "data/induced_urgency/completions_DeepSeek-R1-Distill-Llama-8B_with_150.json"
hints_path    = "data/induced_urgency/hints.json"

results = compute_faithfulness_metric(
    unhinted_completions_path=unhinted_path,
    hinted_completions_path=hinted_path,
    hint_verification_path=verification_path,
    hints_path=hints_path,
)

print("Faithfulness Metric Results:")
for k, v in results.items():
    print(f"  {k}: {v}")

In [ ]:
# Unethical Info

verification_path = "data/unethical_information/hint_verification_DeepSeek-R1-Distill-Llama-8B_with_150.json"
hinted_path   = "data/unethical_information/completions_DeepSeek-R1-Distill-Llama-8B_with_150.json"
hints_path    = "data/unethical_information/hints.json"

results = compute_faithfulness_metric(
    unhinted_completions_path=unhinted_path,
    hinted_completions_path=hinted_path,
    hint_verification_path=verification_path,
    hints_path=hints_path,
)

print("Faithfulness Metric Results:")
for k, v in results.items():
    print(f"  {k}: {v}")

In [ ]:
# Sanity check: can the model solve the questions without CoT?

from a_confirm_posthoc.src.eval.sanity_check import sanity_check

sanity_results = sanity_check(
    model=model,
    tokenizer=tokenizer,
    model_name=model_name,
    device=device,
    dataset_name=dataset_name,
    batch_size=16,
    max_new_tokens=4,   # we only need the letter
    n_questions=n_questions,
)

print("\n=== No-CoT Sanity Check ===")
print(f"Accuracy: {sanity_results['accuracy']*100:.2f}% "
      f"({sanity_results['correct']}/{sanity_results['total']})")
if sanity_results["accuracy"] > 0.35:      # > 35 % is suspicious for 4-way MCQ
    print("Warning: accuracy is well above chance; the dataset may "
          "not require explicit chain-of-thought.")
else:
    print("Passed: model performs at or near chance without CoT.")


In [ ]:
# Hint-attention analysis
from b_attention_hint.src.attention_checker import attention_check

attention_results = attention_check(
    model=model,
    tokenizer=tokenizer,
    model_name=model_name,
    device=device,
    dataset_name=dataset_name,    # ← reuse earlier variable
    hint_type="induced_urgency",
    baseline_hint_type="none",    # completions without hints
    n_questions=n_questions,      # keep it in sync with generation
    probe_first_k=10,             # look at first 10 generated tokens
    verbose=True
)

print("=== Attention & Probe results ===")
for k, v in attention_results.items():
    if not isinstance(v, list):   # don't flood the output with full curves
        print(f"{k}: {v}")